In [1]:
!pip install tensorflow
!pip install opencv-python
!pip install matplotlib
!pip install image

In [2]:
# from tensorflow.python.platform import build_info as tf_build_info
# print(tf_build_info.cuda_version_number)
# # 9.0 in v1.10.0
# print(tf_build_info.cudnn_version_number)
# # 7 in v1.10.0

import tensorflow as tf
print(tf.__version__)

1.12.0


# Imports

In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
# import Image

# if tf.__version__ < '1.4.0':
#     raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [4]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports

In [ ]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Load Model

In [ ]:
# opener = urllib.request.URLopener()
# print(1)
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# print(1)
# MODEL_NAME = 'faster_rcnn_resnet101_kitti_2017_11_08'
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_lowproposals_oid_2017_11_08'
# MODEL_NAME = 'ssd_inception_v2_coco_2017_11_17 (1)'
# MODEL_NAME = 'facessd_mobilenet_v2_quantized_320x320_open_image_v4'
# MODEL_NAME = 'faster_rcnn_resnet101_ava_v2.1_2018_04_30'
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_lowproposals_oid_2018_01_28'
# MODEL_NAME = 'inception_v4_2016_09_09'
# MODEL_NAME = 'mobilenet_v1_0.25_128_quant'
MODEL_NAME = 'mask_rcnn_resnet50_atrous_coco_2018_01_28'

# MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
# MODEL_NAME = 'rfcn_resnet101_coco_2017_11_08'

MODEL_FILE = MODEL_NAME + '.tar.gz'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
# PATH_TO_CKPT = 'inception_v4.ckpt'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90
tar_file = tarfile.open(MODEL_FILE)
# print(1)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
#     if 'inception_v4.ckpt' in file_name:
        tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
namelist=[]
indexlist=[]
for i in category_index.values():
    namelist.append(i['name'])
    indexlist.append(i['id'])

In [ ]:
# # test if video camera works

# # from matplotlib import pyplot as plt
# import cv2
# cap=cv2.VideoCapture(0) # 0 stands for very first webcam attach
# # https://www.youtube.com/watch?v=BUrR6BTx6Mk
# filename="outputtest.avi"#[place were i stored my output file]
# codec=cv2.VideoWriter_fourcc('m','p','4','v')#fourcc stands for four character code
# framerate=30
# resolution=(640,480)

# # item_lost = False
# # item_lost = False
# counter=0   
# tmarker=0
# VideoFileOutput=cv2.VideoWriter(filename,codec,framerate, resolution)
    
# with detection_graph.as_default():
#     with tf.Session(graph=detection_graph) as sess:
#         ret=True
#         while (ret):

#             ret, image_np=cap.read() 

#             VideoFileOutput.write(image_np)
#             cv2.imshow('live_detection',image_np)
#             print("Line hit 2")
#     #             plt.imshow(image_np)
#     #             plt.show()
#             if cv2.waitKey(25) & 0xFF==ord('q'):
#                 break
#                 cv2.destroyAllWindows()
#                 cap.release()

In [ ]:
# execution

import cv2
cap=cv2.VideoCapture(1) # 0 stands for very first webcam attach # 1 is for the Moverio device
filename="outputtest.avi"#[place were i stored my output file]
codec=cv2.VideoWriter_fourcc('m','p','4','v')#fourcc stands for four character code
framerate=30
resolution=(640,480)

item_lost = True
# item_lost = False
counter=0   
tmarker=0
VideoFileOutput=cv2.VideoWriter(filename,codec,framerate, resolution)
    
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        ret=True
        while (ret):

            ret, image_np=cap.read() 

            f = open('output.txt', 'r')
            x = [x.replace("\n", "") for x in f.readlines()][-1]
            for i in range(len(namelist)):
                if namelist[i] in x:
    #                 print(content[i], i)
                    tmarker=indexlist[i]
    #         print(tmarker)

            if tmarker != -1:
                item_lost = True
            if "found" in x:
                tmarker=-1

            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
              # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
              # Visualization of the results of a detection.
            if item_lost:
                image_np = vis_util.lost_item_mode(image_np,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  line_thickness=8,
                  marker=tmarker)
#                 cv2.imwrite(r'C:/Users/user/Desktop/Moverio-MemoryPalace-Instructions/Calhacks-5.0/Object-Detection-On-Live-Video-Feed/models/object_detection/data_images/img_'+str(counter)+'.jpg', image_np)
                counter+=1
            else:
                image_np = vis_util.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  line_thickness=8)
#                 cv2.imwrite(r'C:/Users/user/Desktop/Moverio-MemoryPalace-Instructions/Calhacks-5.0/Object-Detection-On-Live-Video-Feed/models/object_detection/data_images/img_'+str(counter)+'.jpg', image_np)
                counter+=1

            VideoFileOutput.write(image_np)
            cv2.imshow('live_detection',image_np)
            if cv2.waitKey(25) & 0xFF==ord('q'):
                break
                cv2.destroyAllWindows()
                cap.release()